# Linking up Cube Pickers and Plot Customization Options

This is an example of a set of workflow options linked up with ipywidgets.  The options in this demo are for preparing one plot of an iris cube.

There are some assumptions in this workflow:
1. Data path is set to iris.sample_data_path
2. Coordinates for plot axes will be longitude vs latitude
3. Plot type will be line contour (plot customizations are applicable to this plot type)

This notebook demostrates the links between some elements of a selection process.

In [ ]:
import glob
import IPython.display
import ipywidgets
import iris

import cube_browser

In [ ]:
ipynb = IPython.get_ipython()
if ipynb is not None:
    ipynb.magic(u"%matplotlib notebook")
    ipynb.magic(u"%autosave 0")

In [ ]:
# This filepath is a directory containing the set of options for the first selection box.
path = '/home/h06/cbosley/Projects/iris-sample-data/sample_data'
file_options = glob.glob('{}/*'.format(path)) 

# Here is the widget which allows the user to select a file from that directory
files = ipywidgets.Select(
    description='Cube(list)',
    options=file_options,
    width='100%')

# This is the initial list for the next selection box.
# I must define this now in order for it to be changed later upon selection.
cubes = iris.load_raw(files.value)
options = [None] + [cube.standard_name for cube in cubes]

# This defines your next selection box (to choose the cube from your file) 
plots = ipywidgets.Select(
    description='Choose cube',
    options=options,
    width='100%')

# You can see that above, the options in the second box depend on the selection of the first.
# When you change the selection in your first box, you want the options in the second box to change accordingly.
# You must therefore define a function here which changes the values of the options displayed in your second box.
def handle_cubes(sender):
    cubes = iris.load_raw(files.value)
    options = [None] + [cube.standard_name for cube in cubes]
    plots.value = None
    plots.options = options
    plots.value = options[0]

# The following line executes the behaviour in the 'handle_cubes' function upon a value change in the 'files' widget
files.observe(handle_cubes, names='value')  

# Now I want to make a button which will confirm the selection.  This will load the cube and display it.
load_button = ipywidgets.Button(description='Choose this cube')
def pick_cube(sender):
    cube = iris.load_cube(files.value, plots.value)
    print cube
    return cube
load_button.on_click(pick_cube)

# Next I need to offer some plot customizations.  
# The following is a list of widgets which can be used to choose customization options.
colors = ipywidgets.Dropdown(
    options=['None', 'blue', 'green', 'red', 'cyan', 'magenta', 'yellow', 'black', 'white'],
    value='blue',
    description='colors')
cmap = ipywidgets.Dropdown(
    options=['None', 'viridis', 'inferno', 'plasma', 'magma', 'jet', 'summer', 'autumn'],
    value='None',
    description='cmap')
norm = ipywidgets.Dropdown(
    options=['None', 'Autoscale(A)', 'Autoscale_None(A)', 'scaled()'],
    value='None',
    description='norm')
levels = ipywidgets.Text(
    description='levels',
    value='np.linspace(np.min(cube.data), np.max(cube.data), 10)')
origin = ipywidgets.Dropdown(
    options=['None', 'upper', 'lower', 'image'],
    value='None',
    description='origin')
extend = ipywidgets.Dropdown(
    options=['neither', 'both', 'min', 'max'],
    value='both',
    description='extend')
linestyles = ipywidgets.Dropdown(
    options=['None', 'solid', 'dashed', 'dashdot', 'dotted'],
    value='None',
    description='linestyle')

formatting = [colors, cmap, norm, levels, origin, extend, linestyles]

# This button will prompt the appearance of the customization options above.
confirm_button = ipywidgets.Button(description='Customize')
def custom_options(sender):
    for i in range(len(formatting)):
        IPython.display.display(formatting[i])    
confirm_button.on_click(custom_options)

# container is a box with all the selectors I want to display in it.
container1 = ipywidgets.Box(children=[files, plots])
container2 = ipywidgets.Box(children=[load_button, confirm_button])
IPython.display.display(container1, container2)